In [52]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [53]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [54]:
train_dataset = torchvision.datasets.CIFAR10(root='.', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='.', train=False, transform=transforms.ToTensor(), download=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [55]:
print("dataset keys:",train_dataset.__dict__.keys())
print("dataset classes:", train_dataset.classes)
print("dataset data type:", type(train_dataset.data))
print("dataset target type:", type(train_dataset.targets))

dataset keys: dict_keys(['root', 'transform', 'target_transform', 'transforms', 'train', 'data', 'targets', 'classes', 'class_to_idx'])
dataset classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
dataset data type: <class 'numpy.ndarray'>
dataset target type: <class 'list'>


In [56]:
batch_size = 128

dataloader = {
    'train': torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
}

In [57]:
#def block(c_in, c_out, k=5, p=0, s=2, pk=2, ps=1):
def block(c_in, c_out, k=3, p=0, s=2, pk=2, ps=1):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

# def block2(c_in, c_out, k=5, p=3, s=2, pk=2, ps=1):
#     return torch.nn.Sequential(
#         torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
#         torch.nn.ReLU(),
#         torch.nn.MaxPool2d(pk, stride=ps)
#     )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=3, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 784)
    self.conv2 = block(784, 392)
    self.conv3 = block(392, 196)
    self.fc = torch.nn.Linear(196 * 1 * 1, n_outputs)

  # def forward(self, x):
  #   print("Dimensiones:")
  #   print("Entrada: ", x.shape)
  #   x = self.conv1(x)
  #   print("conv1: ", x.shape)
  #   x = self.conv2(x)
  #   print("conv2: ", x.shape)
  #   x = self.conv3(x)
  #   print("conv3: ", x.shape)
  #   x = x.view(x.shape[0], -1)
  #   print("pre fc: ", x.shape)
  #   x = self.fc(x)
  #   print("Salida: ", x.shape)
  #   return x
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [58]:
model = CNN()

output = model(torch.randn(64, 3, 32, 32))

In [59]:
from tqdm import tqdm
import numpy as np

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [61]:
model = CNN()
fit(model, dataloader, epochs=20)

val_loss 2.32584 val_acc 0.14072: 100%|██████████| 79/79 [00:02<00:00, 29.07it/s]


Epoch 1/20 loss 2.03045 val_loss 2.32584 acc 0.27339 val_acc 0.14072


val_loss 2.25356 val_acc 0.19373: 100%|██████████| 79/79 [00:02<00:00, 28.48it/s]


Epoch 2/20 loss 1.75391 val_loss 2.25356 acc 0.36978 val_acc 0.19373


val_loss 2.28089 val_acc 0.19937: 100%|██████████| 79/79 [00:02<00:00, 29.19it/s]


...

Epoch 18/20 loss 1.21300 val_loss 2.34242 acc 0.57948 val_acc 0.22330


val_loss 2.32548 val_acc 0.22538: 100%|██████████| 79/79 [00:02<00:00, 29.68it/s]


Epoch 19/20 loss 1.20064 val_loss 2.32548 acc 0.58432 val_acc 0.22538


val_loss 2.33870 val_acc 0.21924: 100%|██████████| 79/79 [00:02<00:00, 30.46it/s]

Epoch 20/20 loss 1.18781 val_loss 2.33870 acc 0.59154 val_acc 0.21924
